In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import os.path as osp
from pathlib import Path
import json
import time

In [2]:
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HUB_CACHE"] = "cache/models/pretrained"
os.environ["HTTP_PROXY"] = "http://192.168.196.249:18080"
os.environ["HTTPS_PROXY"] = "http://192.168.196.249:18080"

In [3]:
from src.dataset.medical.ddr_dataset import DDRDataset
from src.dataset.medical.pmc_oa_dataset import PMCOADataset

ds = DDRDataset(root_dir="/media/yons/Datasets/OIA-DDR/DDR-dataset/", split="train")
dl = ds.dataloader(batch_size=16, num_workers=16, shuffle=True)

ds = PMCOADataset(root_dir="/media/yons/Datasets/PMC-OA/", split="train")
dl = ds.dataloader(batch_size=16, num_workers=16, shuffle=True)

for batch in dl:
    print(batch)
    break

/home/yons/miniconda3/envs/ntrain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:src.dataset.cache_manager:缓存目录: /home/yons/workspace/sxy/lab/NeuroTrain/cache/datasets/OIA-DDR/v1
INFO:src.dataset.cache_manager:缓存目录: /home/yons/workspace/sxy/lab/NeuroTrain/cache/datasets/PMC-OA/v1


{'image': tensor([[[[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [254, 254, 254,  ..., 255, 255, 255],
          ...,
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255]],

         [[255, 255, 255,  ..., 254, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 254, 254,  ..., 255, 255, 255],
          ...,
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255]],

         [[254, 254, 254,  ..., 255, 255, 255],
          [250, 251, 251,  ..., 254, 255, 255],
          [250, 251, 252,  ..., 251, 253, 255],
          ...,
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255],
          [  0,   0,   0,  ..., 255, 255, 255]]],


        [[[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,

In [ ]:
# 导入LLaVA推理模块
from llava import load_llava_med, llava_med_chat

# 导入Qwen推理模块
from qwen import (
    load_qwen_med,
    load_qwen_vl_med,
    qwen_med_chat,
    qwen_vl_med_chat,
)

device = "cuda:3" if torch.cuda.is_available() else "cpu"
cache_dir = "cache/models/pretrained"

In [ ]:
model, processor = load_qwen_vl_med()

for batch in dl:
    images = batch["image"]
    # 假设 batch size 为 1，如果为多张图片可以遍历
    for img in images:
        # Qwen-VL-Med 输入需要 PIL.Image，如果 img 是 tensor，需要还原
        if isinstance(img, torch.Tensor):
            # 反归一化可以根据你的预处理决定是否需要
            image_np = img.permute(1, 2, 0).cpu().numpy()
            # 如果像素值是0~1，乘255再转为uint8
            if image_np.max() <= 1.0:
                image_np = (image_np * 255).astype("uint8")
            from PIL import Image

            image_pil = Image.fromarray(image_np)
        else:
            image_pil = img
        # 输入模型，假设你的 prompt 是询问对该视网膜图片的诊断
        prompt = "请根据这张视网膜图片进行医学诊断。"
        output_text = qwen_vl_med_chat(
            model, processor, new_text=prompt, new_image=image_pil
        )
        print("模型诊断输出：", output_text)
    break  # 只测试一批

# 输出
# 这张视网膜图片显示了典型的糖尿病视网膜病变特征。糖尿病视网膜病变是一种由糖尿病引起的视网膜疾病，其主要表现为视网膜血管病变和视网膜色素上皮病变。图片中可以看到明显的视网膜血管病变，包括视网膜血管的增生和硬化，以及视网膜色素上皮的病变。这些病变可以导致视力下降和视网膜脱落等严重并发症。糖尿病视网膜病变的诊断需要结合患者的病史、血糖水平、眼底检查结果等多方面的信息。建议患者及时就医，接受规范的治疗和管理。
# 这张视网膜图片显示了典型的糖尿病视网膜病变特征。糖尿病视网膜病变是一种由糖尿病引起的视网膜疾病，其主要表现为视网膜血管病变和视网膜色素上皮病变。图片中可以看到明显的视网膜血管病变，包括视网膜血管的增生和硬化，以及视网膜血管的阻塞和萎缩。此外，还可见到视网膜色素上皮病变，表现为视网膜色素上皮的增生和色素沉着。这些病变的出现是由于糖尿病患者体内血糖水平过高，导致视网膜血管和色素上皮受损。这种病变如果不及时治疗，可能会导致严重的眼部并发症，如视网膜脱落、视力丧失等。因此，对于糖尿病患者来说，定期进行眼部检查和治疗是非常重要的。

`torch_dtype` is deprecated! Use `dtype` instead!
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


模型诊断输出： ['The image shows a detailed view of the retina, with a clear central macula and a well-defined optic disc. The retinal vessels are visible, branching out from the optic disc. There are no obvious signs of hemorrhage, exudates, or other abnormalities in the retina. The overall appearance suggests a healthy retina with no significant pathological findings. However, without clinical context or additional imaging, it is not possible to rule out all possible conditions. A thorough ophthalmic examination by a qualified medical professional is necessary to provide a comprehensive assessment.']
模型诊断输出： ['The image shows a detailed view of the retina, with a clear central macular area and a well-defined optic disc. The retinal vessels are visible, branching out from the optic disc and spreading across the retina. The macular area appears to have a normal appearance, with no obvious signs of pathology such as hemorrhages, exudates, or lesions. The overall retinal architecture appears in

In [6]:
model, tokenizer, processor = load_llava_med()

for batch in dl:
    images = batch["image"]
    # 假设 batch size 为 1，如果为多张图片可以遍历
    for img in images:
        # Qwen-VL-Med 输入需要 PIL.Image，如果 img 是 tensor，需要还原
        if isinstance(img, torch.Tensor):
            # 反归一化可以根据你的预处理决定是否需要
            image_np = img.permute(1, 2, 0).cpu().numpy()
            # 如果像素值是0~1，乘255再转为uint8
            if image_np.max() <= 1.0:
                image_np = (image_np * 255).astype("uint8")
            from PIL import Image

            image_pil = Image.fromarray(image_np)
        else:
            image_pil = img
        # 输入模型，假设你的 prompt 是询问对该视网膜图片的诊断
        prompt = "请根据这张视网膜图片进行医学诊断。"
        output_text = llava_med_chat(
            model, tokenizer, processor, new_text=prompt, new_image=image_pil
        )
        print("模型诊断输出：", output_text)
    break  # 只测试一批

# 输出
#

Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.54s/it]
Some weights of the model checkpoint at microsoft/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 97.94 MiB is free. Including non-PyTorch memory, this process has 23.58 GiB memory in use. Of the allocated memory 23.15 GiB is allocated by PyTorch, and 121.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)